# Dosimetria

Calculadora simples de dosimetria da pena com base na doutrina vigente.

# Como funciona

A pena é calculada na 1ª fase com base nos 8 critérios
do [artigo 59 do CP](https://www.planalto.gov.br/ccivil_03/LEIS/1980-1988/L7209.htm#art59).

Já a 2ª fase leva em consideração os artigos 61...


Definição de constantes base para os cálculos.

In [20]:
WEIGHT_FIRST_STEP = 1 / 8  # Fração de cada critério da primeira fase.

WEIGHT_SECOND_STEP = 1 / 6  # Fração de cada critério da segunda fase.

DAYS_IN_MONTH = 30  # Dias em um mês.

MONTHS_IN_YEAR = 12  # Meses em um ano.

DAYS_IN_YEAR = DAYS_IN_MONTH * MONTHS_IN_YEAR  # Dias em um ano.

Define uma classe de sentença para armazenar e manipular os dados da sentença. 

In [21]:
class Sentence:
    """
    Classe de armazenamento e manipulação de sentenças.
    """

    def __init__(self):
        self.raw_days: int = 0
        self.years: int = 0
        self.months: int = 0
        self.raw_months: int = 0
        self.days: int = 0

    def __update(self):
        """
        Atualiza a sentença com base nos dias absolutos.
        """

        self.days = self.raw_days % DAYS_IN_MONTH
        self.raw_months = self.raw_days // DAYS_IN_MONTH
        self.months = self.raw_months % MONTHS_IN_YEAR
        self.years = self.raw_months // MONTHS_IN_YEAR

    def adjust(self, days: int):
        """
        Ajusta a sentença conforme a quantidade dada de dias.

        :param days: Dias para ajustar a sentença.
        :return: Sentença atualizada.
        """

        self.raw_days += days
        self.__update()
        return self

    def to_str(self, *, raw_days: bool = False, start: bool = True, end: bool = True):
        """
        Converte uma sentença para uma string formatada.

        :param raw_days: Mostrar apenas os dias absolutos.
        :param start: Formatar como início de frase.
        :param end: Formatar como fim de frase.
        :return: String formatada.
        """

        ret_str = ""

        if start:
            ret_str += "A pena é de "

        if raw_days or self.raw_days == 0:
            ret_str += f"{self.raw_days} dias"
        else:
            if self.years:
                ret_str += f"{self.years} anos"
            if self.months and self.days:
                ret_str += (", " if self.years else "") + f"{self.months} meses"
            elif self.months:
                ret_str += (" e " if self.years else "") + f"{self.months} meses"
            if self.days:
                ret_str += (" e " if (self.years or self.months) else "") + f"{self.days} dias"

        return ret_str + ("." if end else "")


Define uma classe de crime, que armazena uma sentença a processa.

In [22]:
class Crime:
    """
    Classe para armazenar e manipular um crime com base em sua dosimetria.
    """

    def __init__(self, *, name: str = None, min_sentence: str, max_sentence: str):
        """
        Cria um crime para dosimetria.
        
        :param name: Nome do crime, opcional.
        :param min_sentence: Pena mínima.
        :param max_sentence: Pena máxima.
        """

        min_amount = int(min_sentence[:-1])
        max_amount = int(max_sentence[:-1])

        self.name = name.strip().lower() if name is not None else None
        self.__evaluated_steps_mask: int = 0
        self.min_sentence_days = (
                min_amount * DAYS_IN_MONTH) if min_sentence[-1] == "m" else min_amount * DAYS_IN_YEAR
        self.max_sentence_days = (
                max_amount * DAYS_IN_MONTH) if max_sentence[-1] == "m" else max_amount * DAYS_IN_YEAR
        self.sentence = Sentence()
        self.sentence.adjust(self.min_sentence_days)
        print(f"Para o crime {f'de {self.name}' if self.name else 'avaliado'}, pena mínima é de "
              f"{self.sentence.to_str(start=False)}")

    def evaluate_first_step(self, first_step_valid_criteria: int):
        """
        Realiza a dosimetria da primeira fase.
        
        :param first_step_valid_criteria: Quantidade de critérios avaliados negativamente na primeira fase.
        """

        if self.__evaluated_steps_mask & 1:
            print(f"{'Crime' if self.name is None else self.name.capitalize()} já avaliado na primeira fase.")
            return

        self.__evaluated_steps_mask |= 1 << 0

        sentence_range_days = self.max_sentence_days - self.min_sentence_days
        criteria_weight_days = sentence_range_days * WEIGHT_FIRST_STEP
        first_step_delta_days = int(criteria_weight_days * max(min(8, first_step_valid_criteria), 0))

        self.sentence.adjust(first_step_delta_days)
        print(f"A pena após valoração da 1ª fase é de {self.sentence.to_str(start=False, end=False)} "
              f"(+{first_step_delta_days / DAYS_IN_MONTH:.1f} meses).")

    def evaluate_second_step(self, *, aggravating_count: int, mitigating_count: int):
        """
        Realiza a dosimetria da segunda fase.
        
        :param aggravating_count: Quantidade de agravantes.
        :param mitigating_count: Quantidade de atenuantes.
        """

        if not self.__evaluated_steps_mask & 1:
            print(f"O {self.name if self.name else 'crime'} ainda não foi avaliado na primeira fase.")
            return

        if self.__evaluated_steps_mask & 1 << 1:
            print(f"{self.name.capitalize() if self.name else 'Crime'} já avaliado na segunda fase.")
            return

        self.__evaluated_steps_mask |= 1 << 1

        criteria_weight_days = self.sentence.raw_days * WEIGHT_SECOND_STEP
        second_step_delta_days = int(criteria_weight_days * (aggravating_count - mitigating_count))

        self.sentence.adjust(second_step_delta_days)
        print(f"A pena após valoração da 2ª fase é de {self.sentence.to_str(start=False, end=False)} "
              f"({'+' if second_step_delta_days >= 0 else ''}{second_step_delta_days / DAYS_IN_MONTH:.1f} meses).")
        

# Utilizando a calculadora
Para realizar um cálculo de dosimetria, o primeiro passo é determinar o intervalo base da sentença para o crime. Para isso, determine abaixo o tempo mínimo da pena e o tempo máximo.
A formatação deve seguir a seguinte forma:

| **Tempo** | **Formatação** |
|:---------:|:--------------:|
|  2 anos   |     `"2a"`     |
|  10 anos  |    `"10a"`     |
|  2 meses  |     `"2m"`     |
| 10 meses  |    `"10m"`     |

Ou seja, deve-se inserir o tempo e logo em seguida um indicador de anos ('*a*') ou meses ('*m*'). Com isso, podemos criar o crime que será avaliado:

In [23]:
minimum_sentence = "6a"  # Pena mínima
maximum_sentence = "20a"  # Pena máxima

test_crime = Crime(min_sentence=minimum_sentence, max_sentence=maximum_sentence)

Para o crime avaliado, pena mínima é de 6 anos.


Agora, na primeira fase da dosimetria devemos indicar quantos, dos 8 critérios, foram avaliados negativamente, e realizamos a avaliação:

In [24]:
first_step_criteria_count = 5  # Critérios avaliados negativamente.

test_crime.evaluate_first_step(first_step_criteria_count)

A pena após valoração da 1ª fase é de 14 anos e 9 meses (+105.0 meses).


Para a segunda fase, devemos indicar quantas agravantes e atenuantes foram contabilizadas, respectivamente, e realizamos a avaliação:

In [25]:
aggravating_factors = 2  # Agravantes
mitigating_factors = 1  # Atenuantes

test_crime.evaluate_second_step(aggravating_count=aggravating_factors, mitigating_count=mitigating_factors)

A pena após valoração da 2ª fase é de 17 anos, 2 meses e 15 dias (+29.5 meses).


Ao fim, temos a dosimetria da pena:

In [26]:
test_crime.sentence.to_str()

'A pena é de 17 anos, 2 meses e 15 dias.'